<h4><font color='red'>Work in Progress</font></h4>

# Reinforcement Learning

In this notebook we are solving the discrete Lunar Landing v2 environment from OpenAI Gym using 1) REINFORCE and 2) Deep Q-learning and compare the two approaches.

**Environment description:**<br>
Landing pad is always at coordinates (0,0). Coordinates are the first two numbers in state vector. Reward for moving from the top of the screen to landing pad and zero speed is about 100..140 points. If lander moves away from landing pad it loses reward back. Episode finishes if the lander crashes or comes to rest, receiving additional -100 or +100 points. Each leg ground contact is +10. Firing main engine is -0.3 points each frame. Solved is 200 points. Landing outside landing pad is possible. Fuel is infinite, so an agent can learn to fly and then land on its first attempt. Four discrete actions available: do nothing, fire left orientation engine, fire main engine, fire right orientation engine.

#### Imports

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from torch.autograd import Variable
from collections import deque, namedtuple
import random
import pickle
import gym
import os
from gym import wrappers
import io
import base64
from IPython.display import HTML

#### Global parameters

In [7]:
CUDA = torch.cuda.is_available()
print('CUDA has been enabled.' if CUDA is True else 'CUDA has been disabled.')

BATCH_SIZE = 32

FloatTensor = torch.cuda.FloatTensor if CUDA else torch.FloatTensor
IntTensor   = torch.cuda.IntTensor if CUDA else torch.IntTensor
LongTensor  = torch.cuda.LongTensor if CUDA else torch.LongTensor
ByteTensor  = torch.cuda.ByteTensor if CUDA else torch.ByteTensor
Tensor      = FloatTensor

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

ENV = 'BreakoutDeterministic-v4'
print(f'\nRunning the {ENV} environment...')

env = gym.make(ENV)
N_OBS_SPACE = env.observation_space.shape[0]
N_ACT_SPACE = env.action_space.n
print(f'State represented by {N_OBS_SPACE}-dimensional vector and {N_ACT_SPACE} actions available.')
del env

CUDA has been enabled.

Running the BreakoutDeterministic-v4 environment...
State represented by 210-dimensional vector and 4 actions available.


#### Save/Load

In [8]:
def save_agent(agent, filename):
    '''
    Saves an agent of specified filename to relative path.
    '''
    path = os.getcwd() + '/agents'
    if not os.path.exists(path):
        os.makedirs(path)
    with open(path + '/' + filename + '.agent', 'wb') as f:
        pickle.dump(agent, f)
    save_stats(agent.test_n, agent.test_r, filename)
    

def load_agent(filename):
    '''
    Loads an agent of specified filename from relative path.
    '''
    with open(os.getcwd() + '/agents' + '/' + filename + '.agent', 'rb') as f:
        return pickle.load(f)
    
    
def save_stats(n, r, filename):
    '''
    Saves stats of specified filename to relative path.
    '''
    path = os.getcwd() + '/agents'
    if not os.path.exists(path):
        os.makedirs(path)
    with open(path + '/' + filename + '.stats', 'wb') as f:
        pickle.dump((n, r), f)

        
def load_stats(filename):
    '''
    Loads stats of specified filename from relative path.
    '''
    with open(os.getcwd() + '/agents' + '/' + filename + '.stats', 'rb') as f:
        return pickle.load(f)

#### Agent base

In [9]:
class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

    
class Agent(object):
    def __init__(self,
                 policy=None,
                 env=gym.make(ENV),
                 num_episodes=1000,
                 discount_factor=0.99,
                 lr=1e-3,
                 test_freq=200,
                 test_num=10,
                 min_reward=-250,
                 max_reward=200):
        super(Agent, self).__init__()
        
        self.env = env
        self.num_episodes = num_episodes
        self.discount_factor = discount_factor
        self.lr = lr
        self.test_freq = test_freq
        self.test_num = test_num
        self.min_reward = min_reward
        self.max_reward = max_reward
        self.achieved_max_reward = False
        #self.rollout_limit = env.spec.timestep_limit
        self.rollout_limit = 10000
        
        
        if policy is not None:
            self.policy = policy.cuda() if CUDA else policy
            self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        
        self.train_n, self.train_r = [], []
        self.test_n, self.test_r = [], []
        self.losses = []
           
    
    def reset_env(self, env=None):
        """
        Resets the current environment using a constant
        seed to make sure environment is deterministic.
        """
        if env is None: env = self.env
        env.seed(0)
        return env.reset()
    
    
    def select_action(self, s):
        """
        Selects an action according to the current policy.
        """
        s = Variable(Tensor(s))
        action_logits = self.policy(s) 
        log_probs = action_logits-torch.logsumexp(action_logits, dim = 1)

        action = torch.distributions.Categorical(logits=action_logits).sample()
        
        return action.data.cpu().numpy(), log_probs[0,action.data.cpu().numpy()]
    
    
    def preprocess(self, x):
        x = torch.tensor(x).permute([2, 0, 1]).data.numpy()
        x = np.mean(x[:, ::2, ::2], axis=0) / 255
        return x.reshape(-1, 1, 105, 80)
    
    def play_episode(self, env=None, replay=False):
        """
        Plays a single episode and returns SAR rollout.
        The logarithm of the action probabilities is also
        included in the rollout (for computing the loss).
        """
        if env is None: env = self.env
        s = self.reset_env(env)
        rollout, eps_r, mapo_traj = [], 0, []
        s = self.preprocess(s)
        for i in range(self.rollout_limit):
            a, log_probs = self.select_action(s)
            s1, r, done, _ = env.step(a)
            s1 = self.preprocess(s1)
            rollout.append((s, a, r, log_probs))
            if done: r += -1
            eps_r += r
            if hasattr(self, 'memory'):
                self.memory.push(Tensor(s), a, Tensor(s1), r)
            if replay: self.replay()
            if eps_r < self.min_reward and env is None: break
            if done: break
            s = s1

        if eps_r > self.max_reward:
            #print('Achieved maximum reward:', eps_r)
            self.achieved_max_reward = True
            
        return np.array(rollout)
    
    
    def compute_loss(self, rewards, log_probs):
        """
        Computes the loss from discounted return.
        """
        
        G, loss = torch.zeros(1,1).type(FloatTensor), 0
        rewards = (rewards-np.mean(rewards))/(np.std(rewards)+1e-05)
        for i in reversed(range(len(rewards))):
            G = self.discount_factor * G + (rewards[i])
            loss = loss - (log_probs[i]*Variable(G))
            #loss = loss - ((log_probs[i]/log_probs[i])*Variable(G))
            
        return loss #/ len(rewards)
    """
        G, loss = torch.zeros(1,1).type(FloatTensor), 0

        rewards = (rewards-np.mean(rewards))/np.std(rewards)
        
        for i in reversed(range(len(rewards))):
            G = self.discount_factor * G + (rewards[i])
            state_val = Variable(G)
            loss = loss - state_val
        
        loss = Variable(loss.data, requires_grad=True)
        return loss 
    """
    
    
    def train(self):
        """
        Runs a full training for defined number of episodes.
        """
        last100 = np.zeros(10)
        for e in range(1, self.num_episodes+1):
            rollout = self.play_episode()
            self.optimize(rollout)

            if e % self.test_freq == 0:
                n, r = self.test()
                print('{:5d},  Reward: {:6.2f},  Length: {:4.2f}'.format(e, r, n))
                last100[(e//10)%10] = r
            if np.mean(last100) >= 200: break
        print('Completed training!')
        self.plot_rewards()


    def test(self):
        """
        Runs a number of tests and computes the
        mean episode length and mean reward.
        """
        n, r = [], []

        for e in range(self.test_num):
            rollout = self.play_episode()     
            rewards = np.array(rollout[:,2], dtype=float)
            n.append(len(rollout))
            r.append(sum(rewards))

        #self.test_n.append(n)
        #self.test_r.append(r)
        #save_agent(self, 'ConvDQN-solved_' + str(len(self.test_n)))
        
        return np.mean(n), np.mean(r)
    
    
    def get_replay(self):
        """
        Renders an episode replay using the current policy.
        """
        env = wrappers.Monitor(self.env, "./gym-results", force=True)
        if hasattr(self, 'epsilon'):
            eps = self.epsilon
            self.epsilon = 0
            self.play_episode(env)
            self.epsilon = eps
        else:
            self.play_episode(env)
        
        
    def plot_rewards(self):
        """
        Plots the moving average of the reward during training.
        """
        def moving_average(a, n=10) :
            ret = np.cumsum(a, dtype=float)
            ret[n:] = ret[n:] - ret[:-n]
            return ret / n
        
        plt.figure(figsize=(16,6))
        plt.subplot(211)
        plt.plot(range(1, len(self.train_r)+1), self.train_r, label='training reward')
        plt.plot(moving_average(self.train_r))
        plt.xlabel('episode'); plt.ylabel('reward')
        plt.xlim((0, len(self.train_r)))
        plt.legend(loc=4); plt.grid()
        plt.subplot(212)
        plt.plot(range(1, len(self.losses)+1), self.losses, label='loss')
        plt.plot(moving_average(self.losses))
        plt.xlabel('episode'); plt.ylabel('loss')
        plt.xlim((0, len(self.losses)))
        plt.legend(loc=4); plt.grid()
        plt.tight_layout(); plt.show()

#### REINFORCE

In [ ]:
class PolicyNet(nn.Module):

    def __init__(self,
                 n_hidden1=32,
                 n_hidden2=32):        
        super(PolicyNet, self).__init__()
        self.fc1 = nn.Linear(N_OBS_SPACE, n_hidden1)
        self.fc2 = nn.Linear(n_hidden1, n_hidden2)
        #self.fc3 = nn.Linear(n_hidden2, n_hidden3)
        self.out = nn.Linear(n_hidden2, N_ACT_SPACE)
        

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return F.softmax(self.out(x), dim=0)

class Conv(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size=3, dilation=1, stride=1, padding=False):
        super(Conv, self).__init__()

        padding = int((kernel_size - 1) / 2) if padding else 0
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, dilation=dilation, stride=stride, padding=padding),
            nn.BatchNorm2d(out_channels),
            #nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.ReLU()
        )

    def forward(self, x):
        return self.conv(x)


class PolicyConvNet(nn.Module):

    def __init__(self, n_out):
        super(PolicyConvNet, self).__init__()

        self.conv1 = Conv(1, 16, kernel_size=8, stride=4)
        self.conv2 = Conv(16, 32, kernel_size=4, stride=2)

        self.fc1 = nn.Linear(2816, 128)
        self.out = nn.Linear(128, n_out)


    def forward(self, x):
        x = x.reshape(-1, 1, 105, 80)
        x = self.conv1(x)
        x = self.conv2(x)
        
        x = x.reshape(-1, 2816)
        x = F.relu(self.fc1(x))
        return self.out(x)
    
    
class REINFORCE(Agent):
    def __init__(self,
                 env=gym.make(ENV),
                 num_episodes=10000,
                 discount_factor=0.99,
                 lr=1e-3,
                 test_freq=250,
                 test_num=10):
        super(REINFORCE, self).__init__(
            policy=PolicyConvNet(n_out = N_ACT_SPACE),
            env=env,
            num_episodes=num_episodes,
            discount_factor=discount_factor,
            lr=lr,
            test_freq=test_freq,
            test_num=test_num)


    def optimize(self, rollout):
        rewards = np.array(rollout[:,2], dtype=float)
        log_probs = np.array(rollout[:,3])
        
        self.optimizer.zero_grad()
        loss = self.compute_loss(rewards, log_probs)
        loss.backward()
        self.optimizer.step()
        #self.scheduler.step()
                   
        self.train_r.append(sum(rewards))
        self.train_n.append(len(rollout)) 
        self.losses.append(loss.detach().cpu())
        
        
agent = REINFORCE(num_episodes=2500,lr = 1e-4, test_freq = 50)
print('start training')
agent.train()
agent.get_replay()
save_agent(agent, 'ConvREINFORCE-solved')

start training
   50,  Reward:   5.00,  Length: 266.70
  100,  Reward:   4.00,  Length: 268.70
  150,  Reward:   5.60,  Length: 283.30
  200,  Reward:   7.80,  Length: 340.40


#### Test

In [ ]:
agent.get_replay()

#### DQN

In [ ]:
class Conv(nn.Module):
    
    def __init__(self, in_channels, out_channels, kernel_size = 3, dilation = 1, padding = False):
        super(Conv, self).__init__()
        
        padding = int((kernel_size-1)/2) if padding else 0
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, dilation=dilation, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.ReLU()
        )
        
    def forward(self, x):
        return self.conv(x)


class PolicyConvNet(nn.Module):
    def __init__(self):        
        super(PolicyConvNet, self).__init__()
        
        self.conv1 = Conv(3, 32)
        self.conv2 = Conv(32, 24)
        self.conv3 = Conv(24, 16)
        
        self.fc1 = nn.Linear(6912, 2048)
        self.fc2 = nn.Linear(2048, 128)
        self.out = nn.Linear(128, N_ACT_SPACE)
        

    def forward(self, x):  
        x = x.reshape(-1, 3, 210, 160)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.reshape(-1, 6912)   
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.out(x)



class PolicyNet(nn.Module):

    def __init__(self,
                 n_hidden1=64,
                 n_hidden2=64):        
        super(PolicyNet, self).__init__()
        self.fc1 = nn.Linear(N_OBS_SPACE, n_hidden1)
        self.fc2 = nn.Linear(n_hidden1, n_hidden2)
        self.out = nn.Linear(n_hidden2, N_ACT_SPACE)
        

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.out(x)


class DQN(Agent):
    def __init__(self,
                 env=gym.make(ENV),
                 num_episodes=10000,
                 discount_factor=0.99,
                 lr=1e-4,
                 test_freq=500,
                 test_num=10,
                 update_target=10,
                 epsilon=1.0,
                 epsilon_decay=0.99993,
                 epsilon_min=0.01,
                 mapo=True):
        super(DQN, self).__init__(
            policy=PolicyConvNet(),
            env=env,
            num_episodes=num_episodes,
            discount_factor=discount_factor,
            lr=lr,
            test_freq=test_freq,
            test_num=test_num)
        
        self.target = PolicyConvNet()
        self.target.load_state_dict(self.policy.state_dict())
        self.target.eval()
        
        self.loss = nn.MSELoss()
        
        if CUDA:
            self.target = self.target.cuda()
        
        self.update_target = update_target
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.memory = ReplayMemory(10000)
        
        
    def select_action(self, s):
        """
        Selects an action according to the current policy.
        """
        if random.random() > self.epsilon:
            with torch.no_grad():
                return int(self.policy(Tensor(s)).argmax().cpu().data.numpy()), None
        else:
            return np.random.randint(N_ACT_SPACE), None
        
    
    def replay(self):
        if len(self.memory) < BATCH_SIZE:
            return
        transitions = self.memory.sample(BATCH_SIZE)
        batch = Transition(*zip(*transitions))
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                              batch.next_state)), dtype=torch.uint8)
        non_final_next_states = torch.cat([Tensor(s) for s in batch.next_state
                                                    if s is not None]).reshape(BATCH_SIZE, -1)

        state_batch = torch.cat(batch.state).reshape(BATCH_SIZE, -1)
        action_batch = torch.cat(torch.split(LongTensor(batch.action), 1)).reshape(BATCH_SIZE, -1)
        reward_batch = Tensor(batch.reward)

        state_action_values = self.policy(state_batch).gather(1, action_batch)

        next_state_values = Tensor(torch.zeros(BATCH_SIZE))
        
        next_state_values[non_final_mask] = self.target(non_final_next_states).max(1)[0].detach()
        expected_state_action_values = ((next_state_values * self.discount_factor) + reward_batch)
        
        loss = self.loss(state_action_values, expected_state_action_values.unsqueeze(1))
        self.optimizer.zero_grad()
        loss.backward()
        for param in self.policy.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()
    
    
    def train(self):
        """
        Runs a full training for defined number of episodes.
        """
        for e in range(1, self.num_episodes+1):
            rollout = self.play_episode(replay=True)

            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay
            
            if e % self.update_target == 0:
                self.target.load_state_dict(self.policy.state_dict())

            if e % self.test_freq == 0:
                n, r = self.test()
                print('{:5d},  Reward: {:6.2f}, , Length: {:4.2f}, Epsilon: {:4.2f}'.format(e, r, n, self.epsilon))
                
            if self.achieved_max_reward: break

        print('Completed training!')
        
    
    def optimize(self, rollout):
        rewards = np.array(rollout[:,2], dtype=float)
        log_probs = np.array(rollout[:,3])
        
        self.optimizer.zero_grad()
        loss = self.compute_loss(rewards, log_probs)
        loss.backward()
        self.optimizer.step()

        #self.train_r.append(sum(rewards))
        #self.train_n.append(len(rollout))
        #self.losses.append(loss.detach().cpu())
        
        
agent = DQN()
agent.train()
agent.get_replay()
save_agent(agent, 'DQN-solved')

#### Test